### JSON config

In [ ]:
{
	"data_sources": {
		"api": {
			"Pollution": "https://api.openaq.org/v1/latest?country=IN&limit=10000",
			"Economy": "https://api.data.gov.in/resource/07d49df4-233f-4898-92db-e6855d4dd94c?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json&offset=0&limit=100"
		},
		"csv": {
			"CryptoMarkets": "crypto-markets.csv"
		}
	}
}

### DataSources

In [ ]:
class Extract:
    
    def __init__(self):
        self.data_sources = json.load(open('data_config.json'))
        self.api = self.data_sources['data_sources']['api']
        self.csv_path = self.data_sources['data_sources']['csv']
    
    
    def getAPISData(self, api_name):
        api_url = self.api[api_name]
        response = requests.get(api_url)
        return response.json()

    
    def getCSVData(self, csv_name):
        csv_data = []
        csv_reader = csv.reader(open(self.csv_path[csv_name], 'r'))
        for row in csv_reader:
            csv_data.append(row)
        return csv_data
    
    
    def databases(self, db_name):
        pass
        

### Transformation

In [ ]:
from DataSources import Extract
from DataLoad import MongoDB
import urllib
import pandas as pd

class Transformation:
    
    def __init__(self, dataSource, dataSet):
        print('Inside')
        extractObj = Extract()
        if dataSource == 'api':
            self.data = extractObj.getAPISData(dataSet)
            funcName = dataSource+dataSet
            getattr(self, funcName)()
        elif dataSource == 'csv':
            self.data = extractObj.getCSVData(dataSet)
            funcName = dataSource+dataSet
            getattr(self, funcName)()
        else:
            self.data = extractObj.databases(dataSet)
    
    
    def apiEconomy(self):
        gdp_india = {}
        for record in self.data['records']:
        # insert into MongoDB like apiPollution
        
        
    def apiPollution(self):
        air_data = self.data['results']
        #
        # there was stuff here
        #
        mongoDB_obj = MongoDB(urllib.parse.quote_plus('root'), urllib.parse.quote_plus('password'), 'host', 'Pollution_Data')
        # Insert Data into MongoDB
        mongoDB_obj.insert_into_db(df, 'Air_Quality_India')

    def csvCryptoMarkets(self):
            print('Please Code me !!!!')

    
if __name__ == '__main__':
    dataSource = input("Please Select the DataSource i.e 'API'/'CSV'/'Database': ").lower()
    print(dataSource)
    dataSet = input('Please select the Data set for Transformation: ').title()
    print(dataSet)
    trans_obj = Transformation(dataSource, dataSet)
    
    

### Main

This is to automate it

In [ ]:
from Transformation import Transformation
import json

class Engine:
    
    def __init__(self, dataSource, dataSet):
        trans_obj = Transformation(dataSource, dataSet)
        
        
if __name__ == '__main__':
    
    etl_data = json.load(open('data_config.json'))
    for dataSource, dataSet in etl_data['data_sources'].items():
        print(dataSource)
        for data in dataSet:
            print(data)
            main_obj = Engine(dataSource, data)

### DataLoad

In [ ]:
from pymongo import MongoClient
import pandas as pd

class MongoDB:

    def __init__(self, user, password, host, db_name ,port='27017', authSource='admin'):
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.db_name = db_name
        self.authSource = authSource
        self.uri = f'mongodb://{self.user}:{self.password}@{self.host}:{self.port}/{self.db_name}?authSource={self.authSource}'
        try:
            self.client = MongoClient(self.uri)
            self.db = self.client[self.db_name]
            print('MongoDB Connection Successful. CHEERS!!!')
        except Exception as e:
            print('Connection Unsuccessful!! ERROR!!')
            print(e)


    def insert_into_db(self, data, collection):
        if isinstance(data, pd.DataFrame):
            try:
                self.db[collection].insert_many(data.to_dict('records'))
                print('Data Inserted Successfully')
            except Exception as e:
                print('OOPS!! Some ERROR Occurred')
                print(e)
            finally:
                self.client.close()
                print('Connection Closed!!!')
        else:
            try:
                self.db[collection].insert_one(data)
                print('Data Inserted Successfully')
            except Exception as e:
                print('OOPS!! Some ERROR Occurred')
                print(e)
            finally:
                self.client.close()
                print('Connection Closed!!!')